### Loading Models

In [20]:
import joblib

#choose the location of the pretrained models
MLP2L=joblib.load(r"MLP2L.pkl")
CNN_MLP2L=joblib.load(r"CNN_2L.pkl")
kNN=joblib.load(r"kNN_k_5.pkl")
RF=joblib.load(r"RF.pkl")

image_folder = r"Unseen_adversarial_images\Adversarial"
fileName="Unseen"

### MLP2L

In [22]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import joblib

# Function to preprocess and predict an image
def preprocess_and_predict(model, img_path):
    # Load the image, resize to 28x28 pixels and convert to grayscale
    img = load_img(img_path, target_size=(28, 28), color_mode='grayscale')
    
    # Convert image to array
    img_array = img_to_array(img)
    
    # Check if the image has already been normalized (pixel values between 0 and 1)
    if img_array.max() > 1:  # This means it has not been normalized
        img_array = img_array / 255.0  # Normalize to range [0, 1]
    
    # Flatten the image to match the model input shape (784,)
    img_array = img_array.flatten()

    # Ensure the image is of shape (1, 784)
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    prediction = MLP2L.predict(img_array)
    
    # Return the predicted class (digit)
    predicted_class = np.argmax(prediction, axis=1)
    return predicted_class[0]

# Get all image paths in the folder
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.png')]

# Initialize lists to store data for the table
originals = []
adversarials = []
predictions = []

# Loop through all the images in the folder and predict
for image_path in image_paths:
    filename = os.path.basename(image_path)
    
    # Split the filename using underscores
    parts = filename.split('_')

    # Extract original value (first occurrence of 'orig')
    orig_value = None
    for part in parts:
        if 'orig' in part:
            orig_value = part.replace('orig', '')  # Extract the number after 'orig'
            break
    
    # Extract adversarial value (last occurrence of 'adv')
    adv_value = None
    for part in reversed(parts):
        if 'adv' in part:
            adv_value = part.replace('adv', '')  # Extract the number after 'adv'
            break
    
    # Ensure the extracted values are valid integers
    try:
        orig_value = int(orig_value) if orig_value else None  # Convert to integer if found
        adv_value = int(adv_value) if adv_value else None    # Convert to integer if found
    except ValueError:
        print(f"Skipping file due to invalid format: {filename}")
        continue
    
    # Make the prediction
    predicted_class = preprocess_and_predict(MLP2L, image_path)
    
    # Append data to the lists
    originals.append(orig_value)  # Store as integer
    adversarials.append(adv_value)  # Store as integer
    predictions.append(predicted_class)

# Create a dataframe to display the results
df = pd.DataFrame({
    'Original': originals,
    'Adversarial': adversarials,
    'Predicted': predictions
})

# Sort the dataframe by the 'Original' column
df = df.sort_values(by='Original')

df.to_excel(f"MLP2L_{fileName}.xlsx")

# Display the table
import ace_tools_open as tools; tools.display_dataframe_to_user(name="Predictions Table", dataframe=df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━

### CNN_MLP2L

In [24]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import joblib

# Function to preprocess and predict an image
def preprocess_and_predict(model, img_path):
    # Load the image, resize to 28x28 pixels and convert to grayscale
    img = load_img(img_path, target_size=(28, 28), color_mode='grayscale')
    
    # Convert image to array
    img_array = img_to_array(img)
    
    # Check if the image has already been normalized (pixel values between 0 and 1)
    if img_array.max() > 1:  # This means it has not been normalized
        img_array = img_array / 255.0  # Normalize to range [0, 1]

    # Ensure the image is of shape (1, 784)
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    prediction = CNN_MLP2L.predict(img_array)
    
    # Return the predicted class (digit)
    predicted_class = np.argmax(prediction, axis=1)
    return predicted_class[0]


# Get all image paths in the folder
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.png')]

# Initialize lists to store data for the table
originals = []
adversarials = []
predictions = []

# Loop through all the images in the folder and predict
for image_path in image_paths:
    filename = os.path.basename(image_path)
    
    # Split the filename using underscores
    parts = filename.split('_')

    # Extract original value (first occurrence of 'orig')
    orig_value = None
    for part in parts:
        if 'orig' in part:
            orig_value = part.replace('orig', '')  # Extract the number after 'orig'
            break
    
    # Extract adversarial value (last occurrence of 'adv')
    adv_value = None
    for part in reversed(parts):
        if 'adv' in part:
            adv_value = part.replace('adv', '')  # Extract the number after 'adv'
            break
    
    # Ensure the extracted values are valid integers
    try:
        orig_value = int(orig_value) if orig_value else None  # Convert to integer if found
        adv_value = int(adv_value) if adv_value else None    # Convert to integer if found
    except ValueError:
        print(f"Skipping file due to invalid format: {filename}")
        continue
    
    # Make the prediction
    predicted_class = preprocess_and_predict(CNN_MLP2L, image_path)
    
    # Append data to the lists
    originals.append(orig_value)  # Store as integer
    adversarials.append(adv_value)  # Store as integer
    predictions.append(predicted_class)

# Create a dataframe to display the results
df = pd.DataFrame({
    'Original': originals,
    'Adversarial': adversarials,
    'Predicted': predictions
})

# Sort the dataframe by the 'Original' column
df = df.sort_values(by='Original')
df.to_excel(f"CNN_MLP2L_{fileName}.xlsx")
# Display the table
import ace_tools_open as tools; tools.display_dataframe_to_user(name="Predictions Table", dataframe=df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━

### kNN

In [26]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Function to preprocess and predict an image
def preprocess_and_predict(model, img_path):
    # Load the image, resize to 28x28 pixels and convert to grayscale
    img = load_img(img_path, target_size=(28, 28), color_mode='grayscale')
    
    # Convert image to array
    img_array = img_to_array(img)
    
    # Check if the image has already been normalized (pixel values between 0 and 1)
    if img_array.max() > 1:  # This means it has not been normalized
        img_array = img_array / 255.0  # Normalize to range [0, 1]
    
    # Flatten the image to match the model input shape (784,)
    img_array = img_array.flatten()

    # Ensure the image is of shape (1, 784)
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    prediction = model.predict(img_array)
    
    # Return the predicted class (digit) directly from the prediction
    predicted_class = prediction[0]  # Since kNN predicts the class directly
    return predicted_class

# Get all image paths in the folder
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.png')]

# Initialize lists to store data for the table
originals = []
adversarials = []
predictions = []


# Loop through all the images in the folder and predict
for image_path in image_paths:
    filename = os.path.basename(image_path)
    
    # Split the filename using underscores
    parts = filename.split('_')

    # Extract original value (first occurrence of 'orig')
    orig_value = None
    for part in parts:
        if 'orig' in part:
            orig_value = part.replace('orig', '')  # Extract the number after 'orig'
            break
    
    # Extract adversarial value (last occurrence of 'adv')
    adv_value = None
    for part in reversed(parts):
        if 'adv' in part:
            adv_value = part.replace('adv', '')  # Extract the number after 'adv'
            break
    
    # Ensure the extracted values are valid integers
    try:
        orig_value = int(orig_value) if orig_value else None  # Convert to integer if found
        adv_value = int(adv_value) if adv_value else None    # Convert to integer if found
    except ValueError:
        print(f"Skipping file due to invalid format: {filename}")
        continue
    
    # Make the prediction
    predicted_class = preprocess_and_predict(kNN, image_path)
    
    # Append data to the lists
    originals.append(orig_value)  # Store as integer
    adversarials.append(adv_value)  # Store as integer
    predictions.append(predicted_class)

# Create a dataframe to display the results
df = pd.DataFrame({
    'Original': originals,
    'Adversarial': adversarials,
    'Predicted': predictions
})

# Sort the dataframe by the 'Original' column
df = df.sort_values(by='Original')
df.to_excel(f"kNN_{fileName}.xlsx")
# Display the table
import ace_tools_open as tools; tools.display_dataframe_to_user(name="Predictions Table", dataframe=df)


Predictions Table


### RF

In [28]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.neighbors import KNeighborsClassifier
import joblib

# Function to preprocess and predict an image
def preprocess_and_predict(model, img_path):
    # Load the image, resize to 28x28 pixels and convert to grayscale
    img = load_img(img_path, target_size=(28, 28), color_mode='grayscale')
    
    # Convert image to array
    img_array = img_to_array(img)
    
    # Check if the image has already been normalized (pixel values between 0 and 1)
    if img_array.max() > 1:  # This means it has not been normalized
        img_array = img_array / 255.0  # Normalize to range [0, 1]
    
    # Flatten the image to match the model input shape (784,)
    img_array = img_array.flatten()

    # Ensure the image is of shape (1, 784)
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    prediction = RF.predict(img_array)
    
    # Return the predicted class (digit) directly from the prediction
    predicted_class = prediction[0]  # Since RF predicts the class directly
    return predicted_class

# Get all image paths in the folder
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.png')]

# Initialize lists to store data for the table
originals = []
adversarials = []
predictions = []


# Loop through all the images in the folder and predict
for image_path in image_paths:
    filename = os.path.basename(image_path)
    
    # Split the filename using underscores
    parts = filename.split('_')

    # Extract original value (first occurrence of 'orig')
    orig_value = None
    for part in parts:
        if 'orig' in part:
            orig_value = part.replace('orig', '')  # Extract the number after 'orig'
            break
    
    # Extract adversarial value (last occurrence of 'adv')
    adv_value = None
    for part in reversed(parts):
        if 'adv' in part:
            adv_value = part.replace('adv', '')  # Extract the number after 'adv'
            break
    
    # Ensure the extracted values are valid integers
    try:
        orig_value = int(orig_value) if orig_value else None  # Convert to integer if found
        adv_value = int(adv_value) if adv_value else None    # Convert to integer if found
    except ValueError:
        print(f"Skipping file due to invalid format: {filename}")
        continue
    
    # Make the prediction
    predicted_class = preprocess_and_predict(RF, image_path)
    
    # Append data to the lists
    originals.append(orig_value)  # Store as integer
    adversarials.append(adv_value)  # Store as integer
    predictions.append(predicted_class)

# Create a dataframe to display the results
df = pd.DataFrame({
    'Original': originals,
    'Adversarial': adversarials,
    'Predicted': predictions
})

# Sort the dataframe by the 'Original' column
df = df.sort_values(by='Original')
df.to_excel(f"RF_{fileName}.xlsx")
# Display the table
import ace_tools_open as tools; tools.display_dataframe_to_user(name="Predictions Table", dataframe=df)


Predictions Table
